# Scraper for [this page](https://www.gutenberg.org/browse/languages/fr)

In [3]:
import os
import re
import re
import json
import random
import pprint
import urllib3
import certifi
from time import sleep
from bs4 import BeautifulSoup
from collections import defaultdict

In [4]:
pp = pprint.PrettyPrinter(indent=2)

In [5]:
url = 'https://www.gutenberg.org/browse/languages/fr'

In [6]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [7]:
r = http.request('GET', url)

In [8]:
soup = BeautifulSoup(r.data, 'html.parser')

In [22]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="ebooks, ebook, books, book, free, online, audio" name="keywords"/>
<meta content="33000+ free ebooks online" name="description"/>
<meta content="public" name="classification"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<script type="application/javascript">
      if (top != self) {
        top.location.replace ('http://www.gutenberg.org');
        alert ('Project Gutenberg is a FREE service with NO membership required. If you paid somebody else to get here, make them give you your money back!');
      }
    </script>
<link href="/css/pg-002.css" rel="stylesheet" type="text/css"/> <link href="/favicon.ico" rel="shortcut icon"/>
<title>Browse By Language: French - Project Gutenberg</title>
</head>
<body>
<div class="menubar">
<div class="logo" id="logo">
<a class="logo" href

In [9]:
h2 = soup.body.find_all('h2')

In [10]:
xa = [i for i in h2 if i.find('a')]
xnoa = [i for i in h2 if not i.find('a')] # contains all the links referring to another section > discarded

---

In [12]:
xnoa[random.randint(0, len(xnoa)-1)]

<h2>Andrieux, Louis</h2>

In [33]:
xnoanext = [i.find_next_sibling('p') for i in x if not i.find('a')]
[x for x in xnoanext if 'See' not in x.get_text()] # they all have a <p> containing the "See..." & other link

[]

---

In [34]:
len(xa)

1527

In [35]:
xaul = [i.find_next_sibling('ul') for i in xa]

In [72]:
data = {}
for li in xaul[1].find_all('li'):
    st = list(li.stripped_strings)
    if 'Author' in st[1]:
        data[st[0]] = li.a['href']
pp.pprint(data)

{ 'Germaine': '/ebooks/18092',
  "Heath's Modern Language Series: La Mère de la Marquise": '/ebooks/22813',
  "L'homme à l'oreille cassée": '/ebooks/13704',
  "Le nez d'un notaire": '/ebooks/51709'}


---

In [11]:
data = {}
for h in h2:
    if h.find('a'):
        author = list(h.stripped_strings)[0]
        name = author[:author.rfind(',')]
        date = author[author.rfind(',')+2:]
        if name not in data.keys():
            data[name] = { 'date' : date }
        for li in h.find_next_sibling('ul').find_all('li'):
            st = list(li.stripped_strings)
            title = ', '.join(st[0:-1])
            meta = st[-1]
            link = li.a['href']
            if 'Author' in meta:
                data[name][title] = link 

In [12]:
pp.pprint(data)

{ "Abbadie, Arnauld d'": { 'Douze ans de séjour dans la Haute-Éthiopie': '/ebooks/18812',
                           'date': '1815-1894?'},
  'About, Edmond': { 'Germaine': '/ebooks/18092',
                     "Heath's Modern Language Series: La Mère de la Marquise": '/ebooks/22813',
                     "L'homme à l'oreille cassée": '/ebooks/13704',
                     "Le nez d'un notaire": '/ebooks/51709',
                     'date': '1828-1885'},
  "Abrantès, Laure Junot, duchesse d'": { "Histoire des salons de Paris (Tome 1/6), Tableaux et portraits du grand monde sous Louis XVI, Le Directoire, le Consulat et l'Empire, la Restauration et le règne de Louis-Philippe Ier": '/ebooks/39331',
                                          "Histoire des salons de Paris (Tome 2/6), Tableaux et portraits du grand monde sous Louis XVI, Le Directoire, le Consulat et l'Empire, la Restauration et le règne de Louis-Philippe Ier": '/ebooks/41121',
                                          "Histoir

In [152]:
it = data.__iter__()

In [201]:
auth = next(it)

In [234]:
titles = [d for d in data[auth] if d is not 'date']
titles

['Belle-Rose',
 'Madame Rose; Pierre de Villerglé',
 "Récits d'un soldat: Une armée prisonnière; Une campagne devant Paris"]

In [303]:
ti = titles.__iter__()

In [306]:
get_ebook(auth, next(ti))

author: Achard, Amédée | title: Madame Rose; Pierre de Villerglé
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN" "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<!--

DON'T USE THIS PAGE FOR SCRAPING.

Seriously. You'll only get your IP blocked.

Read http://www.gutenberg.org/feeds/ to learn how to download Project
Gutenberg metadata much faster than by scraping.

--><html xmlns="http://www.w3.org/1999/xhtml" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:ebook="http://www.gutenberg.org/ebooks/" xmlns:marcrel="http://www.loc.gov/loc.terms/relators/" xmlns:xsd="http://www.w3.org/2001/XMLSchema#" lang="en_US" xml:lang="en_US" version="XHTML+RDFa 1.0">




<head xmlns:og="http://opengraphprotocol.org/schema/" profile="http://a9.com/-/spec/opensearch/1.1/">
<style type="text/css">
.icon   { background: transparent url(/pics/sprite.png?1576694919) 0 0 no-repeat; }
</style>
<link rel="stylesheet" type="text/css" href="/css/pg-d

---

In [103]:
GUTENBERG = 'https://www.gutenberg.org'

In [104]:
ebook = get_ebook("Abrantès, Laure Junot, duchesse d'", "Histoire des salons de Paris (Tome 1/6), Tableaux et portraits du grand monde sous Louis XVI, Le Directoire, le Consulat et l'Empire, la Restauration et le règne de Louis-Philippe Ier")
print(ebook[:100])
print('-'*40)
print(ebook[-100:])


----------------------------------------



In [305]:
def get_ebook(author, title):
    print(f'author: {author} | title: {title}')
    url = GUTENBERG + data[author][title]
    p = http.request('GET', url)
    s = BeautifulSoup(p.data, 'html.parser')
    href = s.find_all(type="text/plain")[0]['href']
    ebook = http.request('GET', url).data.decode('utf-8').replace('\r', '\n')
    print(ebook[:1000])
    
    # clean up Gutenberg header & footer
    title_split = [x for x in re.split(';| |:' ,title.upper()) if x]
    print(f'title modified: {title_split}')
    regex = r'\n *' + ' *\n* *'.join(title_split)
    print(f'regex used: {regex}')    
    title_re = re.compile(regex)
    end_re = re.compile(r'\n* *(FIN(\.| D)|End of the Project Gutenberg EBook) *\n*')
    start_match = re.search(title_re, ebook)
    start_index = start_match.start(0)
    end_match = re.search(end_re, ebook)
    end_index = end_match.start(0)
    
    print(f'start match: {start_match} \nend match: {end_match}')
    print(f'start index: {start_index} | end index: {end_index}')
    print('-'*40)
    print(ebook[:start_index])
    msg = 'END OF CROPPED BEGINNING'
    print('-'*len(msg))
    print(msg)
    print('-'*len(msg))    
    print(ebook[start_index:start_index+100])
    print('\n'+'*'*40+'\n')
    print(ebook[end_index-1000:end_index])    
    msg = 'CROPPED ENDING'
    print('-'*len(msg))
    print(msg)
    print('-'*len(msg))
    print(ebook[end_index:end_index+1000])
    
#     ebook = ebook[start_index:end_index]
#     return ebook

In [298]:
author = "Achard, Amédée"
title = "Belle-Rose"
newurl = GUTENBERG + data[author][title]

p = http.request('GET', newurl)
newsoup = BeautifulSoup(p.data, 'html.parser')
d = newsoup.find_all(type="text/plain")
ebookurl = gut + d[0]['href']
ebookdata = http.request('GET', ebookurl)
ebook = ebookdata.data.decode('utf-8').replace('\r', '\n')

In [299]:
# print(ebookdata.data[:1000])
print(ebook[:1000])

﻿The Project Gutenberg EBook of Belle-Rose, by Amédée Achard



This eBook is for the use of anyone anywhere at no cost and with

almost no restrictions whatsoever.  You may copy it, give it away or

re-use it under the terms of the Project Gutenberg License included

with this eBook or online at www.gutenberg.org





Title: Belle-Rose



Author: Amédée Achard



Release Date: February 20, 2006 [EBook #17808]



Language: French





*** START OF THIS PROJECT GUTENBERG EBOOK BELLE-ROSE ***









Produced by Carlo Traverso and the Online Distributed

Proofreading Team at http://www.pgdp.net (This file was

produced from images generously made available by the

Bibliothèque nationale de France (BnF/Gallica))















                             Belle-Rose



                                Par



                           Amédée Achard







  Nelson

  Éditeurs

  189, rue Saint-Jacques

  Paris.



  Calmann-Lévy

  Éditeurs

  3, rue Auber

  Paris.



  _AMÉDÉE ACHARD

 

In [302]:
title_split = [x for x in re.split(';| |:' ,title.upper()) if x]
print(f'title modified: {title_split}')
regex = r'\n *' + ' *\n* *'.join(title_split)
print(f'regex used: {regex}')
title_re = re.compile(regex)
title_match = re.search(title_re, ebook)
print(title_match)
title_index = title_match.start(0)
print('-'*40)
for o in title_re.finditer(ebook): 
    print(o)
print('-'*40)    
print(ebook[title_index:title_index+500])
print('-'*40)
end_re = re.compile(r'\r*\n* *(FIN\.*|End of the Project Gutenberg EBook) *\n*\r*')
for o in end_re.finditer(ebook): 
    print(o)
print('-'*40)
print(ebook[end_re.search(ebook).start(0):])    

title modified: ['BELLE-ROSE']
regex used: \n *BELLE-ROSE
<_sre.SRE_Match object; span=(2760, 2800), match='\n                             BELLE-ROSE'>
----------------------------------------
<_sre.SRE_Match object; span=(2760, 2800), match='\n                             BELLE-ROSE'>
----------------------------------------

                             BELLE-ROSE









I



LE FILS DU FAUCONNIER





Il y avait, vers l'an 1663, à quelques centaines de pas de Saint-Omer,

une maisonnette assez bien bâtie, dont la porte s'ouvrait sur le grand

chemin de Paris. Une haie vive d'aubépine et de sureau entourait un

jardin où l'on voyait pêle-mêle des fleurs, des chèvres et des enfants.

Une demi-douzaine de poules avec leurs poussins caquetaient dans un coin

entre les choux et les fraisiers; deux ou trois ruches, gr
----------------------------------------
<_sre.SRE_Match object; span=(1001197, 1001212), match='\n\n\n\n\n\nFIN\n\n\n\n\n\n'>
<_sre.SRE_Match object; span=(1001318, 10013

In [106]:
print(ebook[end_re.search(ebook).start(0):])


End of the Project Gutenberg EBook of Histoire des salons de Paris (Tome 1/6), by 
Laure Junot, duchesse d'Abrantès

*** END OF THIS PROJECT GUTENBERG EBOOK HISTOIRE DES SALONS DE PARIS (1/6) ***

***** This file should be named 39331-8.txt or 39331-8.zip *****
This and all associated files of various formats will be found in:
        http://www.gutenberg.org/3/9/3/3/39331/

Produced by Mireille Harmelin, Christine P. Travers and
the Online Distributed Proofreading Team at
http://www.pgdp.net (This file was produced from images
generously made available by the BibliothÃ¨que nationale
de France (BnF/Gallica) at http://gallica.bnf.fr)


Updated editions will replace the previous one--the old editions
will be renamed.

Creating the works from public domain print editions means that no
one owns a United States copyright in these works, so the Foundation
(and you!) can copy and distribute it in the United States without
permission and without paying copyright royalties.  Special rules,
set

---

In [177]:
comma_space = re.compile(r',* ')
t = re.sub(comma_space, '-', list(data.keys())[0]).replace("'", "")
t

'Abbadie-Arnauld-d'

In [171]:
for k, v in data.items():
    print(k)
    print(os.path.isdir(k))
    break

Abbadie, Arnauld d'
False
